# Create Spotify playlist from list

In [79]:
import requests
import json
import re
import pandas as pd

title = "(?P<title>\w+)"
artist = "(?P<artist>\w+)"
holder = "(?P<word>\w+)"

# Generate track id list
def append_id(text):
    """extract and append track id to song_list
    """
    
    track = re.findall(r'"spotify:track:(.*?)"', text)[0]
    if track is not None:
        song_list.append(track)
    return song_list

def spotify_search(term):
    """search for songs on spotify using api and append_id to song_list
    """
    
    if isinstance(term, float):
        return False

    # api request to search
    url = "https://api.spotify.com/v1/search?q="+term+"&type=track"
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.get(url, headers=headers)
    parsed_json = json.loads(r.text)
    
    if parsed_json['tracks']['total'] != 0:
        append_id(r.text)
    else:
        # song unavailable
        return False
    return True


# Generate spotify list
def spotify_add(term):
    """add song to spotify playlist by track id
    """
    # api request to add
    url = "https://api.spotify.com/v1/playlists/"+playlist_id+"/tracks?uris=spotify%3Atrack%3A"+term
    headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json', 'Authorization' : 'Bearer ' + token}
    r = requests.post(url, headers=headers)
    return



def define_pattern(pattern):
#     pattern = "%tracknumber% %artist% %%"

    var_names = re.findall(r'%(.*?)%', pattern)
    for var_name in var_names:
        pattern = re.sub("%" +var_name + "%", "(?P<" + var_name + r'>\\w+)', pattern)
        pattern = pattern.replace(r'(?P<>\w+)', r'(\w+)')

    print(pattern)
    return pattern


# Text processing
def remove_num(text):
    text = re.sub("(\d+)", "", text)
    return text

def remove_sym(text):
    text = re.sub(r'(\W)\1+', r' ', text)
    
    text = text.replace(".", " ").replace("，", "").replace("–", "").replace("*", "").replace(
        "-", " ").replace("·", " ").replace("’", " ").replace("/", " ")
#     text = text.replace("(", " ").replace("（", " ").replace(")", "").replace("）", "")
#     text = text.replace("<", " ").replace(">", "")
#     text = text.replace("《", " ").replace("》", " ")
#     text = text.replace("[", "").replace("]", "").replace("「", "").replace("」", "")
    
    return text

def remove_line(text):
    lines = [line.strip() for line in text.split('\n') if line.strip() != '']
    return lines
   
def split_line(line, pos1=0, pos2=-1):
    songs_to_search = line.split(" ")[pos1] + " " + line.split(" ")[pos2]
    return songs_to_search


def text_process(text, num=True):
    if not num:
        text = remove_num(text)
    text = remove_sym(text)
    
    text = text.replace("[粤]", "").replace("演唱：", "")
    text = text.replace("林隆旋", "林隆璇")
    text = text.replace("眞", "真")
    text = text.replace("\t", " ")
    #text = text.replace("  ", " ")
    lines = remove_line(text)

    return lines

def extract_text(lines, pattern):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        m = songs_to_search[i]
        
        try:
            m = re.match(pattern, m)
            songs_to_search[i] = ' '.join([m[k] for k in m.groupdict()])
        except:
            # the hardcoded way
            print("\n======hardcode======")
            print(songs_to_search[i])
            split_line(songs_to_search[i])
            print(songs_to_search[i])
            
    return songs_to_search


    
# Hard code to split lines
def split_text(lines, pos1=0, pos2=-1):
    songs_to_search = lines
    
    for i in range(len(songs_to_search)):
        songs_to_search[i] = split_line(songs_to_search[i], pos1, pos2)
            
    return songs_to_search


# Web data processing with pandas
def extract_web(link):
    website = pd.read_html(link)
    song_list = []

    all_songs = website[17]
    all_songs

    all_songs.rename(columns=cus_columns, inplace=True)

    if one:
        songs_to_search = all_songs[('十大中文金曲得獎名單（按頒獎次序排序）',  'title')]

    if two:
        all_songs['list1'] = all_songs["title1"] + ' ' +  all_songs["artist1"]
        all_songs['list2'] = all_songs["title2"] + ' ' +  all_songs["artist2"]

        songs_to_search = all_songs["list1"]
        songs_to_search = songs_to_search.append(all_songs["list2"])
        # songs_to_search.drop(index=25, inplace=True)
        
    return songs_to_search

# Extract multiple songs according to artists


# title_between = r'《(.*?)》'
def extract_composer(lines, artist_split="：", title_split="，"):
    lines = [line.strip() for line in text.split('\n') if line.strip() != '']
    songs_to_search = []
    
    for line in lines:
        artist = line.split(artist_split)[0]
        title_str = line.split(artist_split)[1]

        if len(title_split) == 1:
            titles = title_str.split(title_split)
        else:
            titles = re.findall(title_between, title_str)
            
        for title in titles:
            songs_to_search.append(artist + " " + title)
                
    return songs_to_search

# print(songs_to_search)

#### Request for API token.

https://developer.spotify.com/console/post-playlist-tracks/?playlist_id=&position=&uris=

In [29]:
# pattern = "(\w+) " +title+ "--" +artist+ ""
# string = text.splitlines()[0].replace("\t", " ")
string = "呢喃 黃鶯鶯)"
print("String: " + string)
# pattern = title+ " " +artist  # + " " +holder

m = re.match(pattern, string)
m.groupdict()
[m[k] for k in m.groupdict()]
string = ' '.join([m[k] for k in m.groupdict()])
print(string)
# print(m.groupdict().values())

String: 呢喃 黃鶯鶯)


AttributeError: 'NoneType' object has no attribute 'groupdict'

In [129]:
text = """1	–黄莺莺*	留不住的故事	
2	–蔡琴*	最后一夜	
3	–林忆莲*	爱上一个不回家的人	
4	–叶欢*	放我的真心在你的手心	
5	–苏芮*	亲爱的小孩	
6	–王芷蕾*	台北的天空	
7	–张雨生*	我是一颗秋天的树	
8	–王杰*	是否我真的一无所有	
9	–姜育恒*	痛快的歌	
10	–王芷蕾*	翩翩飞起	
11	–张雨生*	一天到晚游泳的鱼	
12	–姜育恒*	天天等天天问"""

playlist_id = '7DZV6oKBobOQVnYS8s89jD'
token = "BQAcg1bfiQotFYh4A6rKr6_I2vXPSCjqJGVoyrWpwWUdiMrewyGRTtWaaS-I-iunv-Ipa5B02jEfF94THYvAw3i-bgevgyJds5uxCyOvPcwQldmDjhNlpesjVR0tP_I-NA80Wn5cWzoe8grFZGw4tieRfMYqpSbWr6ubpX72IVD9HPZyKJS7avFBbvRHs9gYBSfS6Q"

# cus_columns = {"歌曲": "title", "主唱": "artist"}
# cus_columns = {"第一季金曲": "title1", "第二季金曲": "title2", "主唱": "artist1", "主唱.1": "artist2"}
link = None

pattern = "%title% %artist%"

if text:
    pattern = define_pattern(pattern)
    text = re.sub(r'《(.*?)》', r'', text)
    lines = text_process(text, num=False)
    
#     songs_to_search = lines
    songs_to_search = extract_text(lines, pattern)
#     songs_to_search = extract_composer(text, artist_split="——")
    songs_to_search = split_text(lines, 0, 1)
elif link:
    songs_to_search = web_process(link)



(?P<title>\w+) (?P<artist>\w+)


In [130]:
songs_to_search

['黄莺莺 留不住的故事',
 '蔡琴 最后一夜',
 '林忆莲 爱上一个不回家的人',
 '叶欢 放我的真心在你的手心',
 '苏芮 亲爱的小孩',
 '王芷蕾 台北的天空',
 '张雨生 我是一颗秋天的树',
 '王杰 是否我真的一无所有',
 '姜育恒 痛快的歌',
 '王芷蕾 翩翩飞起',
 '张雨生 一天到晚游泳的鱼',
 '姜育恒 天天等天天问']

In [131]:
na = ""
song_list = []

for i in songs_to_search:
    if not spotify_search(i):
        print(" unavailable          ", end="")
        na = na + ", " + i
    print(i)
    
# create playlist and add
for i in song_list:
    spotify_add(i)
    
print("\n缺：" + na[2:])

黄莺莺 留不住的故事
蔡琴 最后一夜
林忆莲 爱上一个不回家的人
叶欢 放我的真心在你的手心
苏芮 亲爱的小孩
王芷蕾 台北的天空
 unavailable          张雨生 我是一颗秋天的树
王杰 是否我真的一无所有
姜育恒 痛快的歌
王芷蕾 翩翩飞起
张雨生 一天到晚游泳的鱼
姜育恒 天天等天天问

缺：张雨生 我是一颗秋天的树


### Custom 

In [118]:
text = """1987.8 黄莺莺《雪在烧》专辑  《你在我灵魂的边缘》 作曲：陈秀男
1987.9 叶欢《因为爱你》专辑  《有谁能明暸》作曲：陈秀男  

1988.2 姜育恒《一世情缘》专辑  制作：陈秀男 姜育恒
1988.4 叶欢《放你的真心在我的手心》专辑  制作：陈秀男
1988.5 合辑《六个朋友》专辑        《不是不在乎你》 作曲：陈秀男
1988.6 忧欢派对《告别17，微笑18》专辑    《如果一定要离去》作曲：陈秀男
1988.6 王芷蕾《悲欢岁月》专辑     《是否依然深爱着我》作曲：陈秀男
1988.7 王杰《忘了你 忘了我》专辑    《你是我胸口永远的痛》作曲：陈秀男
1988.9 姜育恒《跟往事干杯》专辑   制作：陈秀男《今夜星光灿烂》，《每一个辗转不眠的夜》作曲：陈秀男
1988.9 星星‧月亮‧太阳《我有自己的路走》专辑  制作：陈秀男《爱情电话》，《心愿》作曲：陈秀男
1988.10 叶欢《记得我们有约》专辑 制作：陈秀男《多说一些说甜一些》，《不眠的夜》作曲：陈秀男

1989.1 忧欢派对/小虎队《新年快乐》专辑   制作：陈秀男       《新年快乐》 ，《你是我幼稚的情人》作曲：陈秀男
1989.3 合辑《七匹狼》专辑  制作：陈秀男 翁孝良《说在我心中的爱》，《没有》，《永远的蓝天》作曲：陈秀男
1989.4 小虎队《逍遥游》专辑  制作：陈秀男《你的眼睛下着雨》，《我的爱正年轻》，《探险风筝》，《星星宝贝》作曲：陈秀男
1989.6 姜育恒《多年以后·再回首》专辑  制作：陈秀男 姜育恒   《谁能忘了我》作曲：陈秀男
1989.8 王杰《孤星》专辑  制作：陈秀男
1989.8 合辑《1989夏令营》专辑《我的爱我的歌》，《初恋的滋味》作曲：陈秀男
1989.9 小虎队《男孩不哭》专辑 《男孩不哭》作曲：陈秀男
1989.11 叶欢《谁在秋天捡到我的心》专辑 制作：陈秀男《找个地方偷偷想你》，《你的爱烧痛我的心》，《不要皱着眉头看我》作曲：陈秀男
1989.12 姚黛玮《危险的吸引力》专辑     《危险的吸引力》作曲：陈秀男

1990.1 红孩儿《亮出我的年轻护照》专辑 《何去何从》 作曲：陈秀男
1990.1 李之勤《长痛还是短痛》专辑 《如果你的心里有两个人》作曲：陈秀男
1990.1 王杰《向太阳怒吼》专辑  制作：陈秀男 王杰 Ricky Ho
1990.2 小虎队《红蜻蜓》专辑《最爱你哭泣时候的眼睛》，《为了你我会做到》，《骊歌》作曲：陈秀男
1990.5 王杰《我要飞》专辑 《亲爱的你是我最后的爱》作曲：陈秀男
1990.7 陈秀男《弹琴说爱》钢琴演奏专辑系列，一套5张
1990.8 邰正宵《痴心的我》专辑  制作：陈秀男 丁晓雯 《最爱你哭泣时候的眼睛》（翻唱小虎队作品），《阳光住在我心里》作曲：陈秀男
1990.9 小虎队《星星的约会》专辑 《唱我最爱的歌给你听》作曲：陈秀男
1990.9 蔡幸娟《是雨还是你》专辑     《不到天荒情不老》作曲：陈秀男
1990.9 郭富城《对你爱不完》专辑  制作：陈秀男   《说你爱我》，《蓝色眼睛的爱情》作曲：陈秀男
1990.10 金玉岚《迷路的天使》专辑          《金色阳光》作曲：陈秀男
1990.10 方志耀《我是个无心的人》专辑          《我的心》作曲：陈秀男
1990.12 林忆莲《爱上一个不回家的人》专辑 制作：陈秀男 许愿 Dick Lee Makoto Kubota《哈啰寂寞》，《爱伤害了你我》作曲：陈秀男

1991.1 王杰《为了爱 梦一生》专辑 《如果你真的在乎我》 作曲：陈秀男
1991.3 方志耀《留一些空间》专辑 《因为有你》作曲：陈秀男
1991.4 郭富城《我是不是该安静的走开》专辑  制作：陈秀男《我是不是该安静的走开》作曲：陈秀男
1991.4 蔡幸娟《真的让我爱你吗》专辑 《用多少伤心能换回你》作曲：陈秀男
1991.7 王杰《忘记你不如忘记自己》专辑   《流浪的心》作曲：陈秀男
1991.7 林子祥《这次你是真的伤了我的心》专辑  制作：陈秀男 林子祥
1991.8 马萃如《什么样的爱你才会懂》专辑  制作：陈秀男 丁晓雯 《什么样的爱你才会懂》作曲：陈大力/陈秀男，《爱我不爱》作曲：陈秀男
1991.10 郭富城《到底有谁能 够告诉我》专辑  制作：陈秀男《到底有谁能 够告诉我》作曲：陈秀男
1991.10 叶倩文《潇洒走一回》专辑  制作：陈秀男《潇洒走一回》，《我们的爱这么难》作曲：陈大力/陈秀男
1991.11 林忆莲《都市心》专辑  制作：陈秀男 许愿
1991.12 小虎队《再见》专辑  制作：陈秀男 李子恒   《放心去飞》作曲：陈大力/陈秀男，《骊歌（说好这次不掉眼泪）》作曲：陈秀男
1991.12 少女队《我的心要去旅行》专辑     《我的心要去旅行》作曲：陈秀男

1992.1 小虎队《Best Dance Remix》精选专辑  制作：陈大力 陈秀男 李子恒《男孩不哭》，《骊歌II（Farewell Version） 》 作曲：陈秀男
1992.2 郭小霖《相识不相恋》专辑 《留不住》作曲：陈秀男
1992.2 林志颖《不是每个恋曲都有美好回忆》专辑  制作：陈秀男 夏春湧《不是每个恋曲都有美好回忆》作曲：陈大力/陈秀男
1992.2 郭富城《请把我的情感带回家》精选专辑  制作：陈秀男
1992.3 红孩儿《故事》专辑   《你最好永远别爱上我》作曲：陈大力/陈秀男，《王者之师》作曲：陈秀男
1992.4 叶蕴仪《欺骗我的心》专辑  制作：陈秀男 陈耀川 蒋三省 熊美玲        《欺骗我的心》作曲：陈大力/陈秀男
1992.4 林子祥《这样爱过你》专辑  制作：陈秀男 《选择》作曲：陈大力/陈秀男
1992.5 伊能静《游戏》专辑  制作：陈秀男《我不过是你的游戏吗？》，《萤火虫》作曲：陈大力/陈秀男
1992.5 林立洋《负心》专辑        《游子吟》作曲：陈大力/陈秀男
1992.6 合辑《想飞就飞》专辑  制作：陈秀男 李子恒   《想飞就飞》，《旧情人的眼泪》作曲：陈大力/陈秀男
1992.6 林志颖《今年夏天》专辑     《今年夏天》作曲：陈大力/陈秀男

1992.7 王杰《英雄泪》专辑 《黑暗中的哭泣》作曲：陈秀男《希望》，《谁能够爱我到天长地久》作曲：陈大力/陈秀男
1992.7 叶倩文《真心真意过一生》专辑  制作：陈秀男 Tommy Page《真心真意过一生》，《曾经心疼》，《因为寂寞》作曲：陈大力/陈秀男
1992.7 郭富城《爱你》专辑  制作：陈秀男    《爱你》，《不要让你知道》作曲：陈大力/陈秀男，《永远我都会记得》作曲：陈秀男
1992.9 李之勤《想你的夜》专辑       《不该爱的是你》作曲：陈大力/陈秀男
1992.10 叶蕴仪《爱的故事》专辑  制作：陈秀男   《爱的故事》，《是你》，《蓝色爱琴海》作曲：陈大力/陈秀男
1992.11 马萃如《从来没有想过爱上你》专辑  制作：陈秀男   《全心演好每个自己》作曲：陈大力/陈秀男
1992.11 潘越云《心的诱惑》专辑     《来生相守》作曲：陈大力/陈秀男
1992.12 张雨生《大海》专辑          《大海》作曲：陈大力/陈秀男
1992.12 林志颖《为什么受伤的总是我》专辑                   《为什么受伤的总是我》，《我对天空说爱你》，《用我的真心宝贝你的心》作曲：陈大         力/陈秀男
1992.12 苏有朋《我只要你爱我》专辑             《勇气》作曲：陈大力/陈秀男

1993.1 林子祥《决定》专辑  制作：陈秀男 林子祥 《决定》，《重逢》作曲：陈大力/陈秀男
1993.1 王杰《我》专辑  《我》，《无悔无憾》，《回家》作曲：陈大力/陈秀男
1993.1 郭富城《把所有的爱都留给你》专辑  制作：陈秀男     《把所有的爱都留给你》，《情非得已离开你》，《爱到最后》作曲：陈大力/陈秀男
1993.2 蔡幸娟《半点东方心》专辑       《潇洒走一回》作曲：陈大力/陈秀男
1993.3 吴奇隆《追梦》专辑 《追梦》作曲：陈大力/陈秀男
1993.3 王韵禅《明天》专辑 《第一次》作曲：陈大力/陈秀男
1993.3 刘德华《真情难收》专辑  制作：陈秀男 杜自持    《一辈子的错》，《情歌》，《真情难收》作曲：陈大力/陈秀男
1993.4 温兆伦《永远是你》专辑  制作：陈秀男      《永远是你》，《寂寞的人》，《为我守候》作曲：陈大力/陈秀男
1993.4 伊能静《恋爱中的女人》专辑  制作：陈秀男     《天变地变情不变》，《轰轰烈烈去爱》作曲：陈大力/陈秀男，《谁？》作曲：陈秀男
1993.4 叶倩文《明月心》专辑  制作：陈秀男                  《永远》，《93‘潇洒走一回》，         《明月心》，《谁的心能让我停泊》作曲：陈大力/陈秀男
1993.7 苏有朋《等到那一天》专辑 《回到从前》作曲：陈大力/陈秀男
1993.7 林志颖《牵挂你的我》专辑   制作：陈秀男 蔡宗政 夏春湧    《懵懂少年心》，《全新的爱》，《就是这一天》，《戏梦》作曲：陈大力/陈秀男


1993.7 刘德华《我爱你》EP专辑  制作：陈秀男 杜自持   《硬汉子，热女人》作曲：陈大力/陈秀男（《真情难收》粤语版）
1993.8 林子祥《精选之天长地久》专辑  制作：陈秀男 林子祥    《天长地久》，《重逢》，《决定》，《选择》作曲：陈大力/陈秀男
1993.9 刘德华《一生一次》专辑制作：陈秀男 杜自持《一生一次》，《不用人陪》作曲：陈大力/陈秀
1993.10 郭富城《梦难留》专辑  制作：陈秀男《梦难留》，《爱你一生够不够》，《多情总是痛》作曲：陈大力/陈秀男
1993.10 吴奇隆《蓦然回首》专辑 《蓦然回首》，《死心不息》作曲：陈大力/陈秀男
1993.10 王杰《浪子心》精选专辑    《你是我胸口永远的痛》作曲：陈秀男  《回家》，《希望》作曲：陈大力/陈秀男
1993.10 叶欢《鸳鸯锦》精选专辑  制作：吴大卫 陈秀男 李寿全 楼文中《你是我胸口永远的痛》作曲：陈秀男 
1993.12 林子祥《When A Man Loves A Woman》专辑  制作：陈秀男 林子祥
1993.12 小虎队《星光依旧灿烂》专辑 《永远》（翻唱叶倩文作品）作曲：陈大力/陈秀男
1993.12 周葆元《Sorry seems to be the hardest word》专辑  制作：陈秀男      《爱在飘雪的午夜》作曲：陈秀男

1994.1 林志颖《火热的心》专辑 《火热的心》作曲：陈大力/陈秀男
1994.1 张雨生《自由歌》精选专辑   《大海》作曲：陈大力/陈秀男
1994.1 合辑《For Lovers Only》专辑《天长地久》，《永远》，《决定》，《爱的故事》，《我对天空说爱你》，《第一次》作曲：陈大力/陈秀男
1994.1 姜育恒《不朽金曲精选II》专辑   《今夜星光灿烂》作曲：陈秀男
1994.1 郭富城《一生的情人》精选专辑  制作：陈秀男      《把所有的爱都留给你》，《不要让你知道》，《爱你》，《爱到最后》，《梦难留》，《情非得已离开你》作曲：陈大力/陈秀男  《到底有谁能够告诉我》，《我是不是该安静的走开》作曲：陈秀男 
1994.1 叶倩文《离开情人的日子》专辑  制作：陈秀男《人生其实很简单》，《朋友》作曲：陈大力/陈秀男
1994.2 郭富城《梦的尽头就是天涯》专辑 制作：陈秀男  《天涯》，《我的心开始有落寞时候》，《爱你太深》作曲：陈大力/陈秀男  《深情的男人》作曲：陈秀男 
1994.4 吴奇隆《想找一个地方》专辑 《My Summer Dream》作曲：陈大力/陈秀男
1994.5 卓玉《中国少年》专辑 《中国少年》作曲：陈大力/陈秀男

1994.5 王杰《孤星梦》精选专辑 《我》作曲：陈大力/陈秀男
1994.5 李碧华《泪痕心事》专辑 制作：陈秀男     《我的世界》作曲：陈大力/陈秀男
1994.6 郭富城《渴望》专辑  制作：陈秀男  《渴望》，《说声我爱你》作曲：陈大力/陈秀男
1994.7 王杰《候鸟》专辑   《寂寞的人》作曲：陈大力/陈秀男
1994.8 周葆元《总有一天你会懂》专辑 制作：陈秀男《总有一天你会懂》作曲：陈秀男
1994.8 吕方《只想遇到一个人》专辑  制作：陈秀男
1994.9 吴奇隆《双飞》专辑 《飚》作曲：陈大力/陈秀男  
1994.11 叶倩文《潇洒明月心》精选专辑  制作：陈秀男《潇洒走一回》，《曾经心疼》，《明月心》，《选择》，《朋友》作曲：陈大力/陈秀         男
1994.11 叶蕴仪《把我放在心里头》精选专辑  制作：陈秀男  陈耀川 蒋三省《欺骗我的心》，《爱的故事》作曲：陈大力/陈秀男  

1995.1 郭富城《我的开始在这里》专辑  制作：陈秀男《狂恋》，《纠缠一辈子》作曲：陈秀男
1995.8 刘德华《登峰造极精华辑》精选专辑  《真情难收》，《一生一次》，《一辈子的错》作曲：陈大力/陈秀男
"""

In [113]:
# title_between = r'《(.*?)》'
def extract_composer(lines, artist_split="：", title_split="，"):
    songs_to_search = []
    
    for line in lines:
        artist = line.split(artist_split)[0][:3]
#         print(artist)
        title_str = line.split(artist_split)[1]

        if len(title_split) == 1:
            titles = title_str.split(title_split)
        else:
            titles = re.findall(title_between, artist_split + title_str)
#         print(titles)
            
        for title in titles:
            songs_to_search.append(artist + " " + title)
                
    return songs_to_search

In [119]:
text = re.sub("《(.*?)》专辑", "", text)
text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)
lines = [line for line in lines if "《" in line]

songs_to_search = extract_composer(lines, artist_split="《", title_split=r"《(.*?)》")

In [120]:
songs_to_search

['黄莺莺 你在我灵魂的边缘',
 '叶欢  有谁能明暸',
 '合辑  不是不在乎你',
 '忧欢派 如果一定要离去',
 '王芷蕾 是否依然深爱着我',
 '王杰  你是我胸口永远的痛',
 '姜育恒 今夜星光灿烂',
 '星星‧ 爱情电话',
 '叶欢  多说一些说甜一些',
 '合辑  说在我心中的爱',
 '小虎队 你的眼睛下着雨',
 '姜育恒 谁能忘了我',
 '合辑 我的爱我的歌',
 '小虎队 男孩不哭',
 '叶欢  找个地方偷偷想你',
 '姚黛玮 危险的吸引力',
 '李之勤 如果你的心里有两个人',
 '小虎队 最爱你哭泣时候的眼睛',
 '王杰  亲爱的你是我最后的爱',
 '陈秀男 弹琴说爱',
 '邰正宵 最爱你哭泣时候的眼睛',
 '小虎队 唱我最爱的歌给你听',
 '蔡幸娟 不到天荒情不老',
 '郭富城 说你爱我',
 '金玉岚 金色阳光',
 '方志耀 我的心',
 '林忆莲 哈啰寂寞',
 '方志耀 因为有你',
 '郭富城 我是不是该安静的走开',
 '蔡幸娟 用多少伤心能换回你',
 '王杰  流浪的心',
 '马萃如 什么样的爱你才会懂',
 '郭富城 到底有谁能 够告诉我',
 '叶倩文 潇洒走一回',
 '小虎队 放心去飞',
 '少女队 我的心要去旅行',
 '郭小霖 留不住',
 '林志颖 不是每个恋曲都有美好回忆',
 '郭富城 请把我的情感带回家',
 '红孩儿 你最好永远别爱上我',
 '叶蕴仪 欺骗我的心',
 '林子祥 选择',
 '伊能静 我不过是你的游戏吗？',
 '林立洋 游子吟',
 '合辑  想飞就飞',
 '林志颖 今年夏天',
 '叶倩文 真心真意过一生',
 '郭富城 爱你',
 '李之勤 不该爱的是你',
 '叶蕴仪 爱的故事',
 '马萃如 全心演好每个自己',
 '潘越云 来生相守',
 '张雨生 大海',
 '林志颖 为什么受伤的总是我',
 '苏有朋 勇气',
 '王杰  我',
 '郭富城 把所有的爱都留给你',
 '蔡幸娟 潇洒走一回',
 '吴奇隆 追梦',
 '王韵禅 第一次',
 '刘德华 一辈子的错',
 '温兆伦 永远是你',
 '伊能静 天变地变情不变',
 '叶倩文 永远',
 '苏有朋 回到从前',
 '林志颖 懵懂少年心',
 '林子祥 天长地久',
 

In [116]:
lines = [line for line in text.split('\n') if '专辑' not in line]

songs_to_search = []
artist = ""

for line in lines:
    if "《" not in line:
        artist = line
    else:
        match = re.search(r'《(.*?)》', line)
        songs_to_search.append(artist + " " + match.group(1))

In [123]:
text = """一點點愉快，要用一生來感慨。（選自同恩「一個人遺憾」）

也許只有一個人，才能改變這一切。（選自姜育恆「一世情緣」）

苦已無悔，歡也無味，何處尋覓悲歡歲月。（選自王芷蕾「悲歡歲月」）

用我熱血，再一次奮起；就算無名，也頂天立地。（選自蘇芮「鋼鐵的心」）

記得我們有約，約在風雪另一邊。（選自葉歡「記得我們有約」）

兩顆心的交界，你一定會看見，只要你願意走向前。（選自張雨生「天天想你」）

是否我真的一無所有，黑暗之中沈默地探索你的手。（選自王傑「是否我真的一無所有」）

多希望和你同一個星座，唱著同樣的歌。（選自憂歡派對、小虎隊「新年快樂」）

再回首，雲遮斷歸途；再回首，荊棘密佈。（選自姜育恆「再回首」）

曾經在幽幽暗暗、反反覆覆中追問，才知道平平淡淡、從從容容才是真。（選自姜育恆「再回首」）

公寓的窗口，一束寂寞紅；三十歲的我，沿著鐵道走；四壁的沈默，電視的情歌，是不是永遠。（選自知己二重唱「是不是永遠」）

到了下半場的年紀，不必再虛偽地哭泣。（選自鈕大可「唱不老的戀曲」）

說出你的情話，別怕讓風聽到，這裡只有你的愛。（選自小虎隊「逍遙遊」）

我是勇敢的小虎，不流淚的小虎。（選自小虎隊「今天看我」）

誰能忘了我，如果你在乎我，幕落之後，有天我將回來。（選自姜育恆「誰能忘了我」）

有一種生命，不獻給任何人，除了完全付出，不能將它用盡。（選自邰正宵「想飛的心」）

不能不好這樣錯下去，只願只想當它夢一場；誰在秋天撿到我的心，請你把它還給我。（選自葉歡「誰在秋天撿到我的心」）

當愛情走過那條叫做夏日的街，她隱隱約約聽見心中的狂野。（選自陳淑樺「愛情走過夏日街」）

你可以裝作不在意，不去體會我的心情，卻不能要求我必須分享你，別碰我的生活！（選自陳淑樺「我可以」）

說誰的情長，比誰的情多，一定是我，但是我贏了又如何？（選自鍾鎮濤「捨不得」）

青春我已虛度，人間到處迷途。（選自姜育恆「我還有夢」）

跑過昨天的我，跑過心中的山與河，擊敗所有的錯，擊敗命運這個對手。（選自王傑「向太陽怒吼」）

希望對榮耀說，如果吶喊在心中，何不讓它自己決定，它想要去的戰場。（選自黃鶯鶯「讓愛自由」）

多想找人陪我逛，累了睡在馬路上。（選自郭富城「對你愛不完」）

風，為什麼吹了四季還不安寧；路，為什麼刻滿模糊的名字。（選自潘越雲「對你的10個疑問」）

春天來過我這裡，託你帶過信，我深信不疑。（選自潘越雲「春天來過我這裡」）

夢裡的餘溫，夠我抵擋那世間寒冷。（選自王傑「為了愛，夢一生」）

告訴我自己，沒有恨的權利，看淡，看清，看破。（選自王傑「那段血、淚、汗的日子」）

萬里的沙漠，找不到你，愛你愛得像一隻孤鷹。（選自王傑「永遠相信愛情」）

歲月的腳步，沒有人能夠留得住，只希望美好的能最後唱完。（選自姜育恆「當擁抱結束」）

當擁抱已結束，人只有孤獨，期待你在天涯相逢。（選自姜育恆「當擁抱結束」）

生命總在該遇的地方遇，不管你選擇什麼，不管你走或不走。（選自鳳飛飛「生命的每一次遇合」）

紅紅青春敲呀敲，自己的歌唱呀唱。（選自東方快車合唱團「紅紅青春敲呀敲」）

一世的聰明，情願糊塗；一身的遭遇，向誰訴？（選自蔡幸娟「問情」）

愛到不能愛，聚到終須散。（選自蔡幸娟「問情」）

有人乖，有人壞，上天都一一安排。（選自趙傳「忘不了的事，忘不了的人」）

如果你要走，轉身就能走；如果你要酒，一生陪你喝。（選自林子祥「至少還有我在乎」）

如果你倦了，我仍是你的城堡，用整個世界，換一朵你的笑。（選自林子祥「至少還有我在乎」）

沏一壺茶，笑看人世的複雜；一曲唱罷，雨過天青了。（選自姜育恆「多情不悔」）

歲月不知人間，多少的憂傷，何不瀟灑走一回。（選自葉蒨文「瀟灑走一回」）

你的名字才兩行，我的思念卻長。（選自紅孩兒「思念」）

我的眼神和別人不一樣，如果你懂，請你別走。（選自林志穎「不是每個戀曲都有美好回憶」）

日也無所求，暮也無所求，日出日落，只為點一盞不滅的夢。（選自鍾鎮濤、章蓉舫「我的世界只有你最懂」）

千年的命運，容不下個人的惆悵；真愛註定要面對，一層又一層的浪。（選自李翊君「衝不破情網」）

把被騙的感覺洗成黑白照片，提醒自己還有張倔強靈魂。（選自陳明真「到哪裡找那麼好的人」）

你聽過千百首歌，真實的日子還是一個人過。（選自黃品源「留下來陪你生活」）

我的苦，是一張沒有出口的地圖；從清晨醒來就是黃昏，誰也不曾發現我存在。（選自林隆璇「血肉之軀」）

人的一生機會不會太多，勇氣要留到最後一刻鐘。（選自瑪麗亞「擁抱」）

我的苦化作泥土，讓你慢慢綻放，總是開在天涯。（選自郭富城「一次相逢就成永遠」）

無望的淚水，也能發出光輝，一次相逢就成永遠。（選自郭富城「一次相逢就成永遠」）

迷亂的城市，迷亂的人像個孩子，找不到路，回不了家。（選自張學友「相信她，關心她」）

不讓我高飛，強留我一生；不看我眼神，有多苦多恨。（選自周子寒「折磨」）

以為你不知道，我心裡有千百道傷，你不過來我心裡逛了一下，我卻夢想為你撐起一個家。（選自葉璦菱「天變了，情也變了」）

若將我身當作你，就能天天在一起。（選自潘越雲「癡情不是一種罪過」）

不願意一陣風吹過來、一陣風吹過去，愛你。（選自陳雷「往事就是我的安慰」）

我已非我，所以愛上你。（選自趙詠華、陳容「忘我的愛」）

青春不該揮霍，夕陽一點一滴會染紅，如果拿得出一些收穫，請你期待相見那一刻。（選自蘇有朋「天地一沙鷗」）

在城市中學會飛翔是必要的事，輕輕鬆鬆就能夠擁有自己的黃昏。（選自金城武「溫柔超人」）

你是我的天，引我不斷迴旋，操縱我的線，卻又苦苦相連。（選自林良樂「風箏在飛」）

我來自偶然，像一顆塵土，有誰看出我的脆弱。（選自歐陽菲菲「感恩的心」）

感恩的心，感謝命運，花開花落，我一樣會珍惜。（選自歐陽菲菲「感恩的心」）

想像著每個港口，都有收穫，用青春陪自己兜風。（選自趙詠華「我愛大海」）

愛總是這樣，讓戀人輕易下了賭注，也不問到明天，是否還有今天的財富。（選自趙詠華「動心一輩子」）

心很小的時候，世界就變得很小，小得看不起一片美麗的樹葉。（選自藍心湄「祕密」）

心很亂的時候，路就變得很多，我們都是這樣和自己走失。（選自藍心湄「祕密」）

我不必在你眼中找尋一個我，你何必在我眼中找尋一個你？（選自吳百倫「勇敢的人都應該握手」）

我一直以為我是你最不愛的孩子，外面的傷，心裡的苦，絕不願你知道。（選自巫啟賢「父親的來信」）

永遠孤單的兩個人，笑著自以為得意的笑，夢著自以為不醒的夢。（選自高勝美「永遠孤單的兩個人」）

一滴眼淚在臉上，更多喜悅在胸膛，原來到哪裡做我自己都一樣。（選自蘇有朋「覺醒」）

人生變換中有無數浪花，我們永遠在告別。（選自黃品源「交過心的人」）

遠方，如果不能給你希望，記住我的心，還為你留盞燈。（選自黃品源「交過心的人」）

忘記人們殘酷的言語，就當作我能體會他們的感受。（選自趙詠華「問心無愧」）

一次相逢，有太多坎坷。（選自任潔玲「我們有沒有愛過」）

我敢拿起就也能自在的放手，我只想這一輩子忠於自我。（選自張雨生、陶晶瑩「執著」）

有人哭有人笑，有人輸有人老，到結局還不是一樣。（選自呂方「朋友別哭」）

彷彿放棄反而是恩寵，離開才能真正擁有。（選自鄭華娟「成熟」）

不要衰老，不要腐敗，要真正的成熟。（選自鄭華娟「成熟」）

從心裡乾淨的人，外表才令人感動，我不知道自己離那天還要多久？（選自鄭華娟「成熟」）

為什麼我不能更聰明一些、更知足一些，讓我的靈魂不再放縱。（選自任潔玲「明知故犯」）

走過歲月，我才發現世界都不完美；成功或失敗，都有一些錯覺。（選自劉德華「今天」）

滄海有多廣，江湖有多深，局中人才了解。（選自劉德華「今天」）

生命開始，情不情願總要過完一生；交出一片心，不怕被你誤解。（選自劉德華「今天」）

浪子總要回家，在老得沒有夢以前。（選自王傑「有失有得」）

所有的感動啊，別讓時光偷走，人世無情現在看得清。（選自王傑「有失有得」）

有誰不在乎呢，外面風多大，戰鬥後我只想要輕鬆一下。（選自王傑「有失有得」）

像小孩一樣活下去，才能夠平靜我自己。（選自劉漢強「我的幸福還在天涯」）

誰都要一個家，來收藏受的傷，而我的幸福還在天涯。（選自劉漢強「我的幸福留在天涯」）

以為相處只要靠默契，現在才知要勇氣。（選自關德輝「愛到灰心」）

以為真情有用，是完全不了解現實的殘忍。（選自關德輝「太過認真」）

踢著石子走路，就好像不在乎前方的紅燈，而我依然傻傻的，選擇自己的方式生存。（選自關德輝「太過認真」）

別人儘管追逐他們想要的天，我早已放下狂野，挺起自尊。（選自關德輝「太過認真」）

別問我生活累不累，靈魂的事，身體無法解決。（選自曾慶瑜「我背叛了愛」）

我傷過的心，你讓它痊癒，再沒有任何的承諾，如此徹底。（選自李李仁「痊癒」）

希望這一生來無塵去無礙，完全的從容，日月為証。（選自棻蘭「傳說」）

誰知道我的美，被鎖在漆黑的宇宙之中；誰看出我的心，從不曾快樂過。（選自棻蘭「傳說」）

每一顆眼淚都是珍貴的，它不應該為不愛她的人而流。（選自孟庭葦「明天」）

讓內在所有的複雜，醒來吧。（選自周俊偉「太陽」）

哭就哭了，才不怕你看輕，我的自尊像鑽石一樣新。（選自徐懷鈺「失戀布丁」）

淚水當然很傷心，但失戀以後，還是能吃下一客布丁。（選自徐懷鈺「失戀布丁」）

他說，再見後就會遇見更好的人，這裡由溫柔冷靜又直接。（選自鄭秀文「至理名言」）

我還是搞不懂，你如何被詛咒，哪一天還是哪個人，讓你荒蕪心田。（選自游鴻明「荒蕪心田」）

以為斷了這份愛，就能笑著醒過來，太不明白你在我心中，地位無可取代。（選自張惠妹「排山倒海」）

恐懼永遠都會在，只能想著你的臉，記取那燦爛。（選自張惠妹「排山倒海」）

汪洋之中，誰能守護你寂寞的帆？晴空之下，誰來垂憐你眼裡無聲吶喊？（選自張惠妹「排山倒海」）

我決定光速飛出你的眼睛，然後關掉手機、減肥整型、移民火星。（選自信樂團「移民火星」）

趕不上變化，就停下欣賞。（選自林志美、鮑比達「下午茶」）

看開吧，就讓重要的人去趕場，我們一起喝杯下午茶。（選自林志美、鮑比達「下午茶」）

愛過的人，竟變得如此陌生，這樣的緣分非死即傷。（選自戴愛玲「別問我的傷」）

別說那句話，欺騙我們倆；別點那首歌，就讓回憶留著完美形象。（選自戴愛玲「別問我的傷」）

多少前浪死在沙灘，無知後浪還在笑；多少英雄唱完凱歌，才知回不去家鄉。（選自張善為「問天‧問地‧問你」）

容我問天問地問你，誰能判我輸贏，一生燦爛一身孤寂。（選自張善為「問天‧問地‧問你」）

最好心臟夠強，沒人為你禱告，大家都愛拼，只好各取所需、各服各的藥。（選自蕭煌奇「不滿足」）

一切早開始了，一切卻還未輝煌。（選自楊培安「痛也不說出口的我」）

慚愧我曾差點放棄，無愧是你。（選自楊培安「謝謝讓我愛上你」）

如果在你的懷裡甜蜜的死掉，拜託把憔悴的照片都燒掉，最美的一刻要你永遠記得我的好強。（選自徐懷鈺「如果在你的懷裡滿足的死掉」）

你放了太多愛，在我眼裡眉間，可以不用眺望天邊，也能喜悅。（選自蔡淳佳「眼裡眉間」）

有情人不能朝夕相對，也不會傷痛欲絕，自信的人不自憐。（選自蔡淳佳「眼裡眉間」）

忘了最初，心就死了，沒有追求，苦中無法作樂。（選自楊培安「沒有白活」）

擦肩的人，是知音還是過客，沒有掌聲，但請給我自由。（選自楊培安「沒有白活」）

有多值得，就有多難得。有多恩寵，就有多忐忑。疲憊的人生，不管情不情願，都得呼嘯而過。（選自楊培安「沒有白活」）

你說他太狠，太不懂疼女人；可憐的人必有可恨，也許你也有份。（選自戴愛玲、馬玉芬「愛的了解」）

北風中江湖路險炎涼如刀割，恩怨退一步才能參透，我用生死和命運相搏，敢掠奪也能輕輕放手。（選自楊培安「真我最自由」）

當你越飛越高的時候，我終究會看透，這就是命運最美的錯，在召喚著我。（選自陳乃榮「召喚獸」）

老天給人類偶爾一些痛，凡人就得開始做功課。（選自九二一地震十周年紀念曲大合唱「讓愛轉動整個宇宙」）

你天真的透明，讓世俗顯得不可理喻，我只想守護你最後的心情。（選自黃閱「你是恆星」）

我有許多的好朋友，那感覺真是富有，也許我不是某某董，我對你的好你該懂。（選自艾成、符瓊音、謝俊奇「住進你心窩」）

像玫瑰一樣，你輕輕包圍我的一生，沒有埋怨，不能後悔，花瓣用盡了溫存。（選自王柏森「情願做你的天使」）

是愛人幸福，還是被愛完整，答案啊，不如先找到一個愛人。（選自王柏森「情願做你的天使」）

內心滂沱，什麼都痛；骨子裡，盤旋的戰鬥。恩愛，好似繩索。（選自戴愛玲「我愛的人不必愛我」）

想起你一如刀割，不見你又無路可走，你足夠我去懦弱。（選自戴愛玲「我愛的人不必愛我」）

哪怕是一滴稀薄的承諾，抱緊，就笑得很久；笑我，苦的快活。（選自戴愛玲「我愛的人不必愛我」）

就算億萬次失落，你能從深淵默默找回我，讓我洗淨受傷的臉孔，素淨無憂。（選自黃麗星「無憂」）

都是因為我們太好，才被命運阻撓，當初傷害你的毒藥，我自己何嘗不公平服下？（選自鍾鎮濤、周蕙「你對我的好」）

靈魂的傷，刻在肉體忘不掉，因為經過太多浪潮，才被幸運看上。（選自鍾鎮濤、周蕙「你對我的好」）

借了要還，不如賴著，比誰先開口。你看著我，好像羞澀，更多在揣測。（選自炎亞綸「借過」）

散的散，自有理由；背對背，望著黑洞；是否我至少能說，借過你片刻？（選自炎亞綸「借過」）

相遇靠緣分，相對剩偽裝，曾經種下的花，都忘了。（選自許藝娜「等你一句話」）

我等你一句話，再狠狠傷我吧，被毀了，轉過身還可以強大。（選自許藝娜「等你一句話」）

一無所有，又怕什麼，我還有自由。（選自許藝娜「想要愛」）

放我的真心在你的手心，也許明天不再相遇，放你的真情在我的衣襟，風雨吹不進我心的寧靜。（選自「放我的真心在你的手心」）

謝謝賜給我一道傷，人生從此多一種荒涼，這一張網永世不能掙扎，疼啊。（選自「刺傷」）

多希望疲倦的都能一夜好夢，想飛的能安心的降落，人生不該變成災難片的臨摹。（選自「烽火」）

在這個世上如果選一門功課，捍衛了你，我才終於成熟。（選自「說愛你以後」）

你給我的我都記得，我受苦的你不管了，壞掉的承諾，換雙手我撐著。（選自「我都記得」）

多少年我的心跳，總在你身後，氣味頻率光影，都還保管著；你的春天有幾個，我的冬天就幾個，曾經重要的，注定屬於你我。（選自「我都記得」）

都說思念有盡頭，為何盡頭只有我，求一抹微光，足夠照亮以後。（選自「我都記得」）"""

In [124]:
text = remove_num(text)
text = remove_sym(text)

lines = remove_line(text)

songs_to_search = re.findall(r"（選自(.*?)）", text)

In [125]:
songs_to_search

['同恩「一個人遺憾」',
 '姜育恆「一世情緣」',
 '王芷蕾「悲歡歲月」',
 '蘇芮「鋼鐵的心」',
 '葉歡「記得我們有約」',
 '張雨生「天天想你」',
 '王傑「是否我真的一無所有」',
 '憂歡派對、小虎隊「新年快樂」',
 '姜育恆「再回首」',
 '姜育恆「再回首」',
 '知己二重唱「是不是永遠」',
 '鈕大可「唱不老的戀曲」',
 '小虎隊「逍遙遊」',
 '小虎隊「今天看我」',
 '姜育恆「誰能忘了我」',
 '邰正宵「想飛的心」',
 '葉歡「誰在秋天撿到我的心」',
 '陳淑樺「愛情走過夏日街」',
 '陳淑樺「我可以」',
 '鍾鎮濤「捨不得」',
 '姜育恆「我還有夢」',
 '王傑「向太陽怒吼」',
 '黃鶯鶯「讓愛自由」',
 '郭富城「對你愛不完」',
 '潘越雲「對你的個疑問」',
 '潘越雲「春天來過我這裡」',
 '王傑「為了愛夢一生」',
 '王傑「那段血、淚、汗的日子」',
 '王傑「永遠相信愛情」',
 '姜育恆「當擁抱結束」',
 '姜育恆「當擁抱結束」',
 '鳳飛飛「生命的每一次遇合」',
 '東方快車合唱團「紅紅青春敲呀敲」',
 '蔡幸娟「問情」',
 '蔡幸娟「問情」',
 '趙傳「忘不了的事忘不了的人」',
 '林子祥「至少還有我在乎」',
 '林子祥「至少還有我在乎」',
 '姜育恆「多情不悔」',
 '葉蒨文「瀟灑走一回」',
 '紅孩兒「思念」',
 '林志穎「不是每個戀曲都有美好回憶」',
 '鍾鎮濤、章蓉舫「我的世界只有你最懂」',
 '李翊君「衝不破情網」',
 '陳明真「到哪裡找那麼好的人」',
 '黃品源「留下來陪你生活」',
 '林隆璇「血肉之軀」',
 '瑪麗亞「擁抱」',
 '郭富城「一次相逢就成永遠」',
 '郭富城「一次相逢就成永遠」',
 '張學友「相信她關心她」',
 '周子寒「折磨」',
 '葉璦菱「天變了情也變了」',
 '潘越雲「癡情不是一種罪過」',
 '陳雷「往事就是我的安慰」',
 '趙詠華、陳容「忘我的愛」',
 '蘇有朋「天地一沙鷗」',
 '金城武「溫柔超人」',
 '林良樂「風箏在飛」',
 '歐陽菲菲「感恩的心」',
 '歐陽菲菲「感恩的心」',
 '趙詠華「我愛大海」',
 '趙詠華「動心一輩子」',
 '藍心湄「祕密」',
 '藍心湄「祕密